# Example of using the orm classes

Imports

In [1]:
from os import remove
from utils.io import striplog_from_text
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [2]:
remove('./tmp/test_orm_db.db')

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2021-02-01 07:26:38,054 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-01 07:26:38,056 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,061 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-01 07:26:38,062 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,064 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-02-01 07:26:38,066 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,070 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-02-01 07:26:38,071 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,072 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-02-01 07:26:38,074 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,075 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-02-01 07:26:38,077 INFO sqlalchemy.engine.base.Engine ()
2021-02-01

### Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':ROOT_DIR+'/data/test.las', 'F02':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
x = [0., 20.]
y = [0., 120.]

for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    
    strip=striplog_from_text(filename)
    
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, 
                          middle=interval.top.middle,  
                          lower=interval.top.lower, 
                          x=x[bh_id], y=y[bh_id])
        
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, 
                           middle=interval.base.middle,  
                           lower=interval.base.lower, 
                           x=x[bh_id], y=y[bh_id])
        
        d.update({int_id:{'description':interval.description, 
                          'interval_number' : interval_number, 
                          'top': top, 'base': base 
                         }})
        
        interval_number+=1
        int_id += 1
        pos_id += 2

 

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

File /home/yanathan/Projects/GSDMA/data/test.las OK! Creation of the striplog ...
{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f8c1e34e358>, 'base': <core.orm.PositionOrm object at 0x7f8c1e34e320>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f8c1e2fc0f0>, 'base': <core.orm.PositionOrm object at 0x7f8c1e2fc208>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f8c1e2fc2b0>, 'base': <core.orm.PositionOrm object at 0x7f8c1e2fc1d0>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f8c1e2fc358>, 'base': <core.orm.PositionOrm object at 0x7f8c1e2fc278>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f8c1e2fc400>, 'base': <core.orm.PositionOrm object at 0x7f8c1e2fc3c8>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)
p.add_components(components)

2021-02-01 07:26:38,970 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 07:26:38,972 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 07:26:38,973 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 07:26:38,979 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-02-01 07:26:38,980 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-02-01 07:26:38,983 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-01 07:26:39,143 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 07:26:39,147 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 07:26:39,149 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2021-02-01 07:26:39,160 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-02-01 07:26:39,162 INFO sqlalchemy.engine.base.Engine ('F01',)
2021-02-01 07:26:39,171 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-01 07:26:39,173 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.476470588

2021-02-01 07:26:39,436 INFO sqlalchemy.engine.base.Engine (14,)
2021-02-01 07:26:39,438 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,439 INFO sqlalchemy.engine.base.Engine (15,)
2021-02-01 07:26:39,443 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,445 INFO sqlalchemy.engine.base.Engine (16,)
2021-02-01 07:26:39,447 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 07:26:39,532 INFO sqlalchemy.engine.base.Engine (35,)
2021-02-01 07:26:39,537 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,540 INFO sqlalchemy.engine.base.Engine (36,)
2021-02-01 07:26:39,543 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,544 INFO sqlalchemy.engine.base.Engine (37,)
2021-02-01 07:26:39,548 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 07:26:39,887 INFO sqlalchemy.engine.base.Engine (52,)
2021-02-01 07:26:39,890 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,893 INFO sqlalchemy.engine.base.Engine (53,)
2021-02-01 07:26:39,898 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 07:26:39,899 INFO sqlalchemy.engine.base.Engine (54,)
2021-02-01 07:26:39,901 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.commit()

2021-02-01 07:26:40,086 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
p.boreholes[0].intervals[0].description

2021-02-01 07:26:40,170 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 07:26:40,173 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-02-01 07:26:40,175 INFO sqlalchemy.engine.base.Engine ('F01',)
2021-02-01 07:26:40,180 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-01 07:26:40,182 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [14]:
session.close()

2021-02-01 07:26:40,253 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [15]:
p.plot3d(x3d=True)